In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import torch
import open_clip
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePalooza Data/Benchmarking/Beetlepalooza_beetles_image_only.csv"
df = pd.read_csv(file_path, sep="\t")
df = df[['ImageFilePath', 'Genus']]
df.head(2)

,ImageFilePath,Genus
0,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius
1,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius


In [4]:
model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms("ViT-B-32", pretrained="openai")

tokenizer = open_clip.get_tokenizer("ViT-B-32")

device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)

for param in model.parameters() :
    param.requires_grad = False


In [5]:
def extract_clip_features(image_path):
    
    image = Image.open(image_path).convert("RGB")
    
    image_tensor = preprocess_val(image).unsqueeze(0).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image_tensor)
        image_features /= image_features.norm(dim=-1, keepdim=True)

    return image_features.cpu().numpy()
    

In [6]:
X = np.vstack([extract_clip_features(img) for img in tqdm(df["ImageFilePath"], desc="Processing ...") ])

print(f"Extracted feature shape: {X.shape}")

le = LabelEncoder()

y = le.fit_transform(df["Genus"])


df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()


Processing ...: 100%|██████████| 11399/11399 [09:13<00:00, 20.60it/s]

Extracted feature shape: (11399, 512)


In [7]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


In [8]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef


seed = 99


models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "SVMLinear": SVC(kernel="linear"),
    "SVMPolynomial": SVC(kernel="poly", degree=4),
    "SVMRadialBasis": SVC(kernel="rbf", degree=4),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=seed),    
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(256, 128, 32), activation='logistic', alpha=1e-3, learning_rate='adaptive', max_iter=250, random_state=seed)
}

predictions = {}

metrics = {}


for name, model in models.items():
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds

    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)

    metrics[name] = {"Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}

    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


NaiveBayes                | Acc: 53.95% | Prec: 65.84% | Rec: 53.95% | F1: 56.50% | Bal Acc: 61.15% | MCC: 0.5042
LogisticRegression        | Acc: 90.96% | Prec: 90.86% | Rec: 90.96% | F1: 90.82% | Bal Acc: 76.00% | MCC: 0.8979
SVMLinear                 | Acc: 91.01% | Prec: 91.06% | Rec: 91.01% | F1: 90.96% | Bal Acc: 82.99% | MCC: 0.8985
SVMPolynomial             | Acc: 67.41% | Prec: 76.41% | Rec: 67.41% | F1: 66.36% | Bal Acc: 46.06% | MCC: 0.6442
SVMRadialBasis            | Acc: 89.96% | Prec: 89.53% | Rec: 89.96% | F1: 89.29% | Bal Acc: 67.41% | MCC: 0.8865
NearestNeighbor           | Acc: 80.53% | Prec: 80.78% | Rec: 80.53% | F1: 79.87% | Bal Acc: 58.68% | MCC: 0.7801
RandomForest              | Acc: 72.28% | Prec: 74.27% | Rec: 72.28% | F1: 68.51% | Bal Acc: 34.81% | MCC: 0.6846
MLP_Baseline              | Acc: 90.48% | Prec: 90.48% | Rec: 90.48% | F1: 90.31% | Bal Acc: 72.94% | MCC: 0.8927


In [9]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,Genus,Pred_NaiveBayes,Pred_LogisticRegression,Pred_SVMLinear,Pred_SVMPolynomial,Pred_SVMRadialBasis,Pred_NearestNeighbor,Pred_RandomForest,Pred_MLP_Baseline
1766,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Synuchus,Cymindis,Synuchus,Synuchus,Pterostichus,Synuchus,Cratacanthus,Synuchus,Synuchus
2629,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Agonum,Calathus,Calathus,Agonum,Calathus,Calathus,Anisodactylus,Calathus,Calathus


In [10]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BeetlePalooza/12.CLIP-linear-probing-genus.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BeetlePalooza/12.CLIP-linear-probing-genus-metrics.csv", index=False)